In [1]:
#importing all the necessary libraries 
from bs4 import BeautifulSoup
import requests
import time
import datetime
import pandas
import smtplib
import json

In [2]:
#Connecting to the OfferUp website
URL= 'https://offerup.com/search?q=used+cars'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.5'}
webpage = requests.get(URL,headers=headers)



In [3]:
webpage

<Response [200]>

In [4]:
# Defining Variables to be used later in the code.
Soup1 = BeautifulSoup(webpage.content,'html.parser')
Soup2 = BeautifulSoup(Soup1.prettify(),'html.parser')
print(Soup2)


<!DOCTYPE html>

<html lang="en">
<head>
<meta charset="utf-8"/>
<meta content="summary" name="twitter:card"/>
<meta content="@offerup" name="twitter:site"/>
<meta content="245858875457163" property="fb:app_id"/>
<meta content="/assets/images/app-icon/icon-1024x1024.png" property="og:image"/>
<meta content="OfferUp" property="og:site_name"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<meta content="hVckgnfxPSIIYHASW6k-BapqZdaFc19eRe0nI8CneNM" name="google-site-verification"/>
<meta content="f9dac38f246f8c2fa8b2523dbf502a7b" name="p:domain_verify"/>
<link href="/assets/images/app-icon/favicon.png" rel="icon" type="image/png"/>
<link href="/assets/images/app-icon/icon-180x180.png" rel="apple-touch-icon" type="image/png"/>
<title>
   Search: used cars near Richmond, VA - OfferUp
  </title>
<meta content="index,follow" name="robots"/>
<meta content="OfferUp is revolutionizing how we sell by making it a snap! Instantly connect with buyers and sellers near you." 

In [23]:
car_listings = Soup2.find_all('a', class_='jss239 jss240 jss501 jss495')
from bs4 import BeautifulSoup

# Creating a list to store the data
data = []

# Iterating through each car listing
for listing in car_listings:
    # Extracting information from the listing
    title = listing['title']

    # Getting the aria-label attribute, handling cases where it may not be present
    aria_label = listing.get('aria-label', '')
    
    # Extracting price, location, and vehicle miles
    price_start = aria_label.find('$')
    if price_start != -1:
        price_end = aria_label.find(' ', price_start)
        price = aria_label[price_start + 1:price_end].replace(',', '')

    location_start = aria_label.find('in ')
    if location_start != -1:
        location = aria_label[location_start + 3:]

    # Extracting vehicle miles based on the 'k miles' format
    miles_start = aria_label.find('k miles')
    if miles_start != -1:
        miles_end = aria_label.find(' ', miles_start)
        vehicle_miles = aria_label[miles_start - 3:miles_end].replace(',', '')
    else:
        # If 'k miles' is not found, extracting the miles in the '99k miles' format
        miles_start_k = aria_label.find('k miles')
        miles_start_miles = aria_label.find(' miles')

        if miles_start_k != -1:
            miles_end = aria_label.find(' ', miles_start_k)
            vehicle_miles = aria_label[miles_start_k - 3:miles_end].replace(',', '')
        elif miles_start_miles != -1:
            miles_end = aria_label.find(' ', miles_start_miles)
            vehicle_miles = aria_label[miles_start_miles - 3:miles_end].replace(',', '')
        else:
            # If neither 'k miles' nor ' miles' is found, I've set vehicle_miles to an empty string or another default value
            vehicle_miles = ""

    # Appending the information to the data list
    data.append([title, price, location, vehicle_miles])





In [24]:
Today = datetime.date.today()

# Define the CSV file path
csv_file_path = 'C:\\Users\\rpsstudent\\Desktop\\my projects\\Amazon Web Scraping\\offer_up_used_car_listings.csv'

with open(csv_file_path, 'w', newline='', encoding='UTF8') as csv_file:
    csv_writer = csv.writer(csv_file)
    
    # Creating the header
    csv_writer.writerow(['Title', 'Price', 'Location', 'Vehicle Miles', 'Date'])
    
    # Inserting the data
    csv_writer.writerows(data)

print(f'Data has been exported to {csv_file_path}')

Data has been exported to C:\Users\rpsstudent\Desktop\my projects\Amazon Web Scraping\offer_up_used_car_listings.csv


In [ ]:
import schedule
import time

def scrape_and_export_data():
    # Define the URL and headers
    URL = 'https://offerup.com/search?q=used+cars'
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
        'Accept-Language': 'en-US,en;q=0.5'
    }

    # Connecting to the website
    webpage = requests.get(URL, headers=headers)
    Soup1 = BeautifulSoup(webpage.content, 'html.parser')
    Soup2 = BeautifulSoup(Soup1.prettify(), 'html.parser')

    # Finding car listings
    car_listings = Soup2.find_all('a', class_='jss239 jss240 jss501 jss495')

    # Creating a list to store the data
    data = []

    # Iterating through each car listing
    for listing in car_listings:
        # Extract information from the listing
        title = listing['title']

        # Get the aria-label attribute, handling cases where it may not be present
        aria_label = listing.get('aria-label', '')

        # Extract price, location, and vehicle miles
        price_start = aria_label.find('$')
        if price_start != -1:
            price_end = aria_label.find(' ', price_start)
            price = aria_label[price_start + 1:price_end].replace(',', '')
        else:
            price = ""

        location_start = aria_label.find('in ')
        if location_start != -1:
            location = aria_label[location_start + 3:]
        else:
            location = ""

        # Extracting vehicle miles based on the 'k miles' format
        miles_start = aria_label.find('k miles')
        if miles_start != -1:
            miles_end = aria_label.find(' ', miles_start)
            vehicle_miles = aria_label[miles_start - 3:miles_end].replace(',', '')
        else:
            # If 'k miles' is not found, extracting the miles in the '99k miles' format
            miles_start_k = aria_label.find('k miles')
            miles_start_miles = aria_label.find(' miles')

            if miles_start_k != -1:
                miles_end = aria_label.find(' ', miles_start_k)
                vehicle_miles = aria_label[miles_start_k - 3:miles_end].replace(',', '')
            elif miles_start_miles != -1:
                miles_end = aria_label.find(' ', miles_start_miles)
                vehicle_miles = aria_label[miles_start_miles - 3:miles_end].replace(',', '')
            else:
                # If neither 'k miles' nor ' miles' is found, set vehicle_miles to an empty string or another default value
                vehicle_miles = ""

        # Append the information to the data list
        data.append([title, price, location, vehicle_miles])

    # Get the current date
    today = datetime.date.today()

    # Define the CSV file path
    csv_file_path = 'C:\\Users\\rpsstudent\\Desktop\\my projects\\Amazon Web Scraping\\offer_up_used_car_listings.csv'

    # Write the data to the CSV file
    with open(csv_file_path, 'w', newline='', encoding='UTF8') as csv_file:
        csv_writer = csv.writer(csv_file)

        # Creating the header
        csv_writer.writerow(['Title', 'Price', 'Location', 'Vehicle Miles', 'Date'])

        # Inserting the data
        csv_writer.writerows(data)

    print(f'Data has been exported to {csv_file_path}')

# Scheduling  the function to run every first of the month at a specific time
schedule.every().day.at("12:00").do(scrape_and_export_data).tag('monthly_task')

while True:
    schedule.run_pending()
    time.sleep(30 * 24 * 60 * 60)
